## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in Toronto. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the _k_-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Download and Explore Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in Toronto</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>

5.  <a href="#item5">Examine Clusters</a>  
    </font>
    </div>


Before we get the data and start exploring it, let's download all the dependencies that we will need.


<B>download all the dependencies that we will need. </B>
<H3> WARNING! loading these libraries will take several minutes! </H3>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes  # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c anaconda beautifulsoup4 -y
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

#import warnings
#warnings.simplefilter('ignore')
print('Libraries imported.')


Libraries imported.


## 1. Download and Explore Dataset


<H1> PART 1 </H1>

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 


#### Load and explore the data


Next, let's load the data.


In [2]:
# Loads the html into a pandas dataframe
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
Toronto_table = pd.read_html(link,header=0,flavor='html5lib')[0]
print(Toronto_table.head())

                                     M1ANot assigned  \
0                    M1BScarborough(Malvern / Rouge)   
1  M1CScarborough(Rouge Hill / Port Union / Highl...   
2  M1EScarborough(Guildwood / Morningside / West ...   
3                             M1GScarborough(Woburn)   
4                          M1HScarborough(Cedarbrae)   

                    M2ANot assigned  \
0                   M2BNot assigned   
1                   M2CNot assigned   
2                   M2ENot assigned   
3                   M2GNot assigned   
4  M2HNorth York(Hillcrest Village)   

                            M3ANorth York(Parkwoods)  \
0                      M3BNorth York(Don Mills)North   
1     M3CNorth York(Don Mills)South(Flemingdon Park)   
2                                    M3ENot assigned   
3                                    M3GNot assigned   
4  M3HNorth York(Bathurst Manor / Wilson Heights ...   

                  M4ANorth York(Victoria Village)  \
0  M4BEast York(Parkview Hill / Woodbi

Let's take a quick look at the data.


In [3]:
Toronto_table.shape

(19, 9)

In [4]:
Toronto_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 9 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   M1ANot assigned                                   19 non-null     object
 1   M2ANot assigned                                   19 non-null     object
 2   M3ANorth York(Parkwoods)                          19 non-null     object
 3   M4ANorth York(Victoria Village)                   19 non-null     object
 4   M5ADowntown Toronto(Regent Park / Harbourfront)   19 non-null     object
 5   M6ANorth York(Lawrence Manor / Lawrence Heights)  19 non-null     object
 6   M7AQueen's Park(Ontario Provincial Government)    19 non-null     object
 7   M8ANot assigned                                   19 non-null     object
 8   M9AEtobicoke(Islington Avenue)                    19 non-null     object
dtypes: object(9)
memory usage: 1.5+ KB


<B> After removing header info on import, the first row of data came in as column names; parse the column names into revised data frame</B>

In [5]:
column_names = ['PostalCode','Borough', 'Neighborhood']
neighborhoods = pd.DataFrame(columns=column_names)
clmn = Toronto_table.columns.tolist()
#
# Iterating the list of column names
for i in clmn:
    PostalCode = i[:3] # parse the zipcode prefix
    length=len(i)
    Rtlen = length - 3 # string length after zipcode prefix  
    BandN = str(i[-Rtlen:])
    l1 = str(i[-Rtlen:]).split('(', 1)  #split the borough from neighborhood
    Borough = l1[0]
    if len(l1)>1 :
        length=len(l1[1])-1
        l2=str(l1[1]).split(')',1) #remove final parenthisis from neighborhood string
        Neighborhood = l2[0]
        #print( PostalCode + " " + Borough + " " + Neighborhood)     
    else:
        Neighborhood=Borough
    df2 = (pd.DataFrame.from_records([{'PostalCode':PostalCode,'Borough':Borough,'Neighborhood':Neighborhood}]))
    #print(df2)
    neighborhoods = pd.concat([neighborhoods,df2])
print(neighborhoods.head())

  PostalCode           Borough                Neighborhood
0        M1A      Not assigned                Not assigned
0        M2A      Not assigned                Not assigned
0        M3A        North York                   Parkwoods
0        M4A        North York            Victoria Village
0        M5A  Downtown Toronto  Regent Park / Harbourfront


Now convert remaining rows into revised dataframe

In [6]:
# Convert each column of dataframe to it's own row and parse it into 3 columns


colcount=0
rowcount = 0
#for index, row in Toronto_table.iterrows(): 
for rowcount in range(len(Toronto_table)):
    for colcount in range(len(Toronto_table.columns)):
        df1=Toronto_table.iloc[rowcount:rowcount+1,colcount:colcount+1] 
        i=str(df1.iloc[0,0])
        #print(i)
        PostalCode = i[:3] # parse the zipcode prefix
        #print(PostalCode)
        length=len(i)
        Rtlen = length - 3 # string length after zipcode prefix  
        BandN = str(i[-Rtlen:])
        l1 = str(i[-Rtlen:]).split('(', 1)  #split the borough from neighborhood
        Borough = l1[0]
        if len(l1)>1 :
            length=len(l1[1])-1
            l2=str(l1[1]).split(')',1) #remove final parenthisis from neighborhood strig
            Neighborhood = l2[0]
            #print( PostalCode + " " + Borough + " " + Neighborhood)     
        else:
            Neighborhood=Borough
        df2 = (pd.DataFrame.from_records([{'PostalCode':PostalCode,'Borough':Borough,'Neighborhood':Neighborhood}]))
        #print(df2)
        neighborhoods = pd.concat([neighborhoods,df2])
neighborhoods.shape

(180, 3)

<B> Complete Dataframe by storing only the zip codes assigned to a borough to the Dataframe table_assigned </B>

In [7]:
# Extracting Table that has an assigned borrow
table_assigned = neighborhoods[neighborhoods['Borough'] != 'Not assigned']
# remove trailing parenthesis from Neighborhood

# Extracting Table that has not assigned borrow
table_not_assigned = neighborhoods[neighborhoods['Borough'] == 'Not assigned']

##More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, 
##you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
##These two rows will be combined into one row with the neighborhoods separated with a comma 
##Below code not needed as they are already combined, separated by a slash as shown below

#table_assigned.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
#table_assigned = table_assigned[['PostalCode','Borough','Neighborhood']].drop_duplicates()

print(neighborhoods[neighborhoods.PostalCode == 'M5A'] )

  PostalCode           Borough                Neighborhood
0        M5A  Downtown Toronto  Regent Park / Harbourfront


In [8]:
# Display row counts and column counts
table_assigned.shape

(103, 3)

<H1> End Part 1 </H1>

<H1> PART 2 </H1>

<B> Create a new dataframe containing Latitude and Longitude for Toronto Postal Codes</B>


In [9]:
df_data_1 = pd.read_csv(r"Geospatial_Coordinates.csv")
print(df_data_1.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [10]:
# New pandas dataframe
column_names = ['PostalCode','Borough', 'Neighborhood', 'Latitude', 'Longitude']
df_data_1.columns = ['PostalCode', 'Latitude', 'Longitude']
# merging with inner join because records without latitude / Longitude are of no use to the below analyses
merged_neighborhood = neighborhoods.merge(df_data_1, how='inner', left_on=['PostalCode'], right_on=['PostalCode']) 
merged_neighborhood.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PostalCode    103 non-null    object 
 1   Borough       103 non-null    object 
 2   Neighborhood  103 non-null    object 
 3   Latitude      103 non-null    float64
 4   Longitude     103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [11]:
print(merged_neighborhood[merged_neighborhood['PostalCode'] == 'M5A'])

  PostalCode           Borough                Neighborhood  Latitude  \
2        M5A  Downtown Toronto  Regent Park / Harbourfront  43.65426   

   Longitude  
2 -79.360636  


In [12]:
# Display row counts and column counts in merged dataframe
merged_neighborhood.shape

(103, 5)

<H1> End Part 2 </H1>


<H1> PART 3 </H1>

<B> Cluster Analysis, working only with boroughs containing the word 'Toronto' </B>


In [13]:
# subset dataframe to only ones with Toronto in borough column
Toronto_neighborhood = merged_neighborhood[merged_neighborhood['Borough'].str.contains("Toronto", case=False)]
print(Toronto_neighborhood.head())

   PostalCode           Borough                Neighborhood   Latitude  \
2         M5A  Downtown Toronto  Regent Park / Harbourfront  43.654260   
9         M5B  Downtown Toronto    Garden District, Ryerson  43.657162   
15        M5C  Downtown Toronto              St. James Town  43.651494   
19        M4E      East Toronto                 The Beaches  43.676357   
20        M5E  Downtown Toronto                 Berczy Park  43.644771   

    Longitude  
2  -79.360636  
9  -79.378937  
15 -79.375418  
19 -79.293031  
20 -79.373306  


In [15]:
Toronto_neighborhood.shape

(39, 5)

In [16]:
# get the latitude and longitude of toronto to draw a map 

address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create the map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_neighborhood['Latitude'], Toronto_neighborhood['Longitude'], Toronto_neighborhood['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [17]:
VERSION = '20210317' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 1000
CLIENT_ID = 'xxxx' ## Foursquare ID removed; to run, enter your own FourSquare Client ID
CLIENT_SECRET = 'xxxx' ## Foursquare Client Secret removed; to run, enter your own FourSquare Client Secret

<B> Let's explore a single neighborhood in our dataframe. </B>


Get the neighborhood's name.


In [19]:
print(Toronto_neighborhood.head())

   PostalCode           Borough                Neighborhood   Latitude  \
2         M5A  Downtown Toronto  Regent Park / Harbourfront  43.654260   
9         M5B  Downtown Toronto    Garden District, Ryerson  43.657162   
15        M5C  Downtown Toronto              St. James Town  43.651494   
19        M4E      East Toronto                 The Beaches  43.676357   
20        M5E  Downtown Toronto                 Berczy Park  43.644771   

    Longitude  
2  -79.360636  
9  -79.378937  
15 -79.375418  
19 -79.293031  
20 -79.373306  


<B> Now, let's get the top 100 venues that are in the neighborhood of The Beaches, East Toronto witin 500 meters. </B>


First, let's create the GET request URL. Name your URL **url**.


In [20]:
# type your answer here
neighborhood_latitude = 43.676357
neighborhood_longitude = -79.293031
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        lat, 
        lng, 
        VERSION, 
        radius, 
        LIMIT)
#url # display URL


Send the GET request and examine the resutls


In [21]:
results = requests.get(url).json()
print(results)

{'meta': {'code': 200, 'requestId': '6054b9d400649b6429d5bafe'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Leslieville', 'headerFullLocation': 'Leslieville, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 16, 'suggestedBounds': {'ne': {'lat': 43.667243904500005, 'lng': -79.31534912291306}, 'sw': {'lat': 43.6582438955, 'lng': -79.32776687708693}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '58d4650e2f91cb5ccec03447', 'name': 'Rorschach Brewing Co.', 'location': {'address': '1001 Eastern Ave', 'crossStreet': 'Woodfield Rd', 'lat': 43.6634831695922, 'lng': -79.31982368639481, 'labeledLatLngs': [{'label': 'display', 'lat': 43.6634831695922, 'lng': -79.31982368639481}], 'distance': 162, 'postalCode':

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '58d4650e2f91cb5ccec03447',
   'name': 'Rorschach Brewing Co.',
   'location': {'address': '1001 Eastern Ave',
    'crossStreet': 'Woodfield Rd',
    'lat': 43.6634831695922,
    'lng': -79.31982368639481,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.6634831695922,
      'lng': -79.31982368639481}],
    'distance': 162,
    'postalCode': 'M4L 1A8',
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['1001 Eastern Ave (Woodfield Rd)',
     'Toronto ON M4L 1A8',
     'Canada']},
   'categories': [{'id': '50327c8591d4c4b30a586d5d',
     'name': 'Brewery',
     'pluralName': 'Breweries',
     'shortName': 'Brewery',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/brewery_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-24-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Rorschach Brewing Co.,Brewery,43.663483,-79.319824
1,Leslieville Farmers Market,Farmers Market,43.664901,-79.319784
2,The Sidekick,Comic Shop,43.664484,-79.325162
3,Chino Locos,Burrito Place,43.664653,-79.325584
4,Queen Margherita Pizza,Pizza Place,43.664685,-79.324164


And how many venues were returned by Foursquare?


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


<a id='item2'></a>


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
Toronto_neighborhood.head

<bound method NDFrame.head of     PostalCode                                            Borough  \
2          M5A                                   Downtown Toronto   
9          M5B                                   Downtown Toronto   
15         M5C                                   Downtown Toronto   
19         M4E                                       East Toronto   
20         M5E                                   Downtown Toronto   
24         M5G                                   Downtown Toronto   
25         M6G                                   Downtown Toronto   
30         M5H                                   Downtown Toronto   
31         M6H                                       West Toronto   
35         M4J                              East YorkEast Toronto   
36         M5J                                   Downtown Toronto   
37         M6J                                       West Toronto   
41         M4K                                       East Toronto   
42  

In [33]:
toronto_venues = getNearbyVenues(names=merged_neighborhood['Neighborhood'],
                                   latitudes=merged_neighborhood['Latitude'],
                                   longitudes=merged_neighborhood['Longitude']
                                  )
print(toronto_venues.head())

       Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2  Victoria Village              43.725882              -79.315572   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1           Variety Store       43.751974       -79.333114   
2  Victoria Village Arena       43.723481       -79.315635   
3               Portugril       43.725819       -79.312785   
4             Tim Hortons       43.725517       -79.313103   

          Venue Category  
0                   Park  
1      Food & Drink Shop  
2           Hockey Arena  
3  Portuguese Restaurant  
4            Coffee Shop  


And let's examine the new dataframe size.


In [38]:
print(toronto_venues.shape)


(2127, 7)


In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [40]:
toronto_venues.groupby('Neighborhood').count()
print('There are {} unique venue categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 unique venue categories.


#### Let's confirm the new size


In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0000,0.0

#### Let's print each neighborhood along with the top 5 most common venues


In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4  Middle Eastern Restaurant  0.00


----Alderwood / Long Branch----
            venue  freq
0     Pizza Place  0.25
1     Coffee Shop  0.12
2    Skating Rink  0.12
3  Sandwich Place  0.12
4             Pub  0.12


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                       Bank  0.09
1                Coffee Shop  0.09
2                   Pharmacy  0.04
3                      Diner  0.04
4  Middle Eastern Restaurant  0.04


----Bayview Village----
                 venue  freq
0                 Café  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4        Metro Station  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [44]:
# loops through grouped venues 
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4  Middle Eastern Restaurant  0.00


----Alderwood / Long Branch----
            venue  freq
0     Pizza Place  0.25
1     Coffee Shop  0.12
2    Skating Rink  0.12
3  Sandwich Place  0.12
4             Pub  0.12


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                       Bank  0.09
1                Coffee Shop  0.09
2                   Pharmacy  0.04
3                      Diner  0.04
4  Middle Eastern Restaurant  0.04


----Bayview Village----
                 venue  freq
0                 Café  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4        Metro Station  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.head())

                                        Neighborhood 1st Most Common Venue  \
0                                          Agincourt                Lounge   
1                            Alderwood / Long Branch           Pizza Place   
2  Bathurst Manor / Wilson Heights / Downsview North                  Bank   
3                                    Bayview Village                  Café   
4                 Bedford Park / Lawrence Manor East    Italian Restaurant   

       2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0  Latin American Restaurant        Breakfast Spot          Skating Rink   
1                   Pharmacy                   Pub                   Gym   
2                Coffee Shop   Fried Chicken Joint    Frozen Yogurt Shop   
3        Japanese Restaurant                  Bank    Chinese Restaurant   
4                Coffee Shop        Sandwich Place          Liquor Store   

  5th Most Common Venue 6th Most Common Venue        7th Most Common Venue

<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [47]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [112]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = merged_neighborhood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M4A,North York,Victoria Village,43.725882,-79.315572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Theater,Wine Shop,Farmers Market
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [52]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Ethiopian Restaurant
1,Alderwood / Long Branch,Pizza Place,Pharmacy,Pub,Gym,Sandwich Place,Coffee Shop,Skating Rink,Dessert Shop,Dim Sum Restaurant,Diner
2,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Fried Chicken Joint,Frozen Yogurt Shop,Restaurant,Supermarket,Sushi Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Deli / Bodega
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,Bedford Park / Lawrence Manor East,Italian Restaurant,Coffee Shop,Sandwich Place,Liquor Store,Juice Bar,Indian Restaurant,Restaurant,Sushi Restaurant,Café,Pub
5,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Farmers Market,Pharmacy,Seafood Restaurant,Beer Bar,Cheese Shop,Bistro
6,Birch Cliff / Cliffside West,College Stadium,Skating Rink,Café,General Entertainment,Farm,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Escape Room
7,Brockton / Parkdale Village / Exhibition Place,Café,Breakfast Spot,Bakery,Coffee Shop,Furniture / Home Store,Burrito Place,Stadium,Italian Restaurant,Bar,Intersection
8,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Coffee Shop,Plane,Harbor / Marina,Bar,Airport Gate,Airport Food Court
9,Caledonia-Fairbanks,Women's Store,Park,Pool,Bar,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


In [54]:
# add clustering labels
neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop('Cluster Labels', 1)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = merged_neighborhood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Pizza Place,Portuguese Restaurant,Hockey Arena,French Restaurant,Intersection,Coffee Shop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Electronics Store
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,2.0,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Theater,Gym / Fitness Center,Farmers Market
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,2.0,Coffee Shop,Diner,Sushi Restaurant,Park,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Restaurant
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,0.0,Fast Food Restaurant,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,2.0,Gym,Restaurant,Coffee Shop,Beer Store,Bike Shop,Chinese Restaurant,Discount Store,Japanese Restaurant,Dim Sum Restaurant,Café
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937,2.0,Pizza Place,Athletics & Sports,Gastropub,Intersection,Flea Market,Pharmacy,Café,Breakfast Spot,Bank,Gym / Fitness Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2.0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Café,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Hotel,Pizza Place,Lingerie Store


Finally, let's visualize the resulting clusters


In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [121]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0.0,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Theater,Wine Shop,Farmers Market
9,Downtown Toronto,0.0,Clothing Store,Coffee Shop,Café,Hotel,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Italian Restaurant,Ramen Restaurant
15,Downtown Toronto,0.0,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Gastropub,Clothing Store,Seafood Restaurant,Farmers Market,Restaurant,Department Store
20,Downtown Toronto,0.0,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Pharmacy,Seafood Restaurant,Restaurant,Farmers Market,Beer Bar,Shopping Mall
24,Downtown Toronto,0.0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Thai Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Poke Place
25,Downtown Toronto,0.0,Grocery Store,Café,Park,Nightclub,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Dumpling Restaurant
30,Downtown Toronto,0.0,Coffee Shop,Café,Restaurant,Clothing Store,Bakery,Gym,Deli / Bodega,Hotel,Thai Restaurant,Bookstore
31,West Toronto,0.0,Bakery,Pharmacy,Grocery Store,Pool,Music Venue,Middle Eastern Restaurant,Café,Brewery,Bar,Bank
35,East YorkEast Toronto,0.0,Pizza Place,Intersection,Park,Convenience Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
36,Downtown Toronto,0.0,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Sporting Goods Shop,Italian Restaurant,Scenic Lookout,Brewery,Fried Chicken Joint


#### Cluster 2


In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Park,Food & Drink Shop,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
52,North York,1.0,Park,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
61,Central Toronto,1.0,Bus Line,Park,Swim School,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
64,York,1.0,Park,Jewelry Store,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
66,North York,1.0,Convenience Store,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store
85,Scarborough,1.0,Playground,Intersection,Park,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
91,Downtown Toronto,1.0,Park,Playground,Trail,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


#### Cluster 3


In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2.0,Pizza Place,Portuguese Restaurant,Hockey Arena,French Restaurant,Intersection,Coffee Shop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Electronics Store
2,Downtown Toronto,2.0,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Theater,Gym / Fitness Center,Farmers Market
3,North York,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store
4,Queen's Park,2.0,Coffee Shop,Diner,Sushi Restaurant,Park,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Restaurant
7,North York,2.0,Gym,Restaurant,Coffee Shop,Beer Store,Bike Shop,Chinese Restaurant,Discount Store,Japanese Restaurant,Dim Sum Restaurant,Café
8,East York,2.0,Pizza Place,Athletics & Sports,Gastropub,Intersection,Flea Market,Pharmacy,Café,Breakfast Spot,Bank,Gym / Fitness Center
9,Downtown Toronto,2.0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Café,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Hotel,Pizza Place,Lingerie Store
10,North York,2.0,Pizza Place,Bakery,Sushi Restaurant,Japanese Restaurant,Falafel Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
12,Scarborough,2.0,Bar,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farm
13,North York,2.0,Gym,Restaurant,Coffee Shop,Beer Store,Bike Shop,Chinese Restaurant,Discount Store,Japanese Restaurant,Dim Sum Restaurant,Café


#### Cluster 4


In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,3.0,Bakery,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farm


#### Cluster 5


In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4.0,Baseball Field,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Farm
101,Etobicoke,4.0,Baseball Field,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Farm


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) and [Polong Lin](https://www.linkedin.com/in/polonglin?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). I hope you found this lab interesting and educational. Feel free to contact us if you have any questions!


This notebook is part of a course on **Coursera** called _Applied Data Science Capstone_. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description         |
| ----------------- | ------- | ------------- | -------------------------- |
| 2020-11-26        | 2.0     | Lakshmi Holla | Updated the markdown cells |
|                   |         |               |                            |
|                   |         |               |                            |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
